# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
df = pd.read_csv(url, index_col=0)
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
df.shape

(6000, 61)

In [3]:
X = df.drop(['url', 'Popular'], axis=1)
y = df['Popular']

In [4]:
y.mean()

0.5

In [5]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression

# Create a dictionary that contains both models
models = {'lr': LogisticRegression(),
          'dt': DecisionTreeRegressor(),
         }   

In [7]:
# Train all the models
for model in models.keys():
    models[model].fit(X_train, y_train)

In [8]:
# predict test for each model
y_pred = pd.DataFrame(index=y_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

In [9]:
# Evaluate each model by MSE
from sklearn.metrics import mean_squared_error

for model in models.keys():
    print('mse',model,np.sqrt(mean_squared_error(y_pred[model], y_test)))

mse lr 0.6082762530298219
mse dt 0.6757711644237764


In [10]:
# Evaluate each model by Accuracy and F1 score

from sklearn import metrics

for model in models.keys():
    print('f1',model,metrics.f1_score(y_pred[model], y_test))
    print('Acurracy',model,metrics.accuracy_score(y_pred[model], y_test))

f1 lr 0.6127006280530356
Acurracy lr 0.63
f1 dt 0.5368492224475998
Acurracy dt 0.5433333333333333


# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [11]:
# Estimate 300 bagged samples

n_estimators = 300
np.random.seed(123)
n_samples = X_train.shape[0]

# create bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(n_estimators)]

#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import BaggingClassifier

#bagreg = BaggingClassifier(DecisionTreeClassifier(max_depth = None), n_estimators=100, bootstrap=True, random_state=1)
#bagreg.fit(X_train, y_train)
#y_pred = bagreg.predict(X_test)
#print(y_pred)
#print('f1', metrics.f1_score(y_test, y_pred))

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression 

seeds = np.random.randint(1, 10000, size=n_estimators)
trees = {}

# 100 decision trees where max_depth=None
for i in range(0,100):
    trees[i] = DecisionTreeClassifier(max_features="sqrt", max_depth=None, random_state=seeds[i])
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

# 100 Decision Trees where max_depth=2
for i in range(100,200):
    trees[i] = DecisionTreeClassifier(max_features="sqrt", max_depth=2, random_state=seeds[i])
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

# 100 Logistic Regressions    
for i in range(200,300):
    trees[i] = LogisticRegression(random_state=seeds[i])
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

In [13]:
# Predict the estimators

# Create an empty DataFrame as big as much estimators are
y_pred_df = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))

# Predict and fill the DF results matrix
for i in range(n_estimators):
    y_pred_df.iloc[:, i] = trees[i].predict(X_test)

y_pred_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
1483,0,1,0,1,0,1,1,1,0,0,...,1,1,1,1,1,1,1,1,1,1
2185,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2520,1,1,0,0,0,1,0,0,1,0,...,1,1,1,1,1,1,1,1,1,1
3721,1,1,1,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3727,0,1,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [14]:
# For each observation, sum the columns
y_pred_df.sum(axis=1)[:5]

1483    252
2185    252
2520    215
3721    261
3727     46
dtype: int64

In [15]:
from sklearn import metrics

# Ensemble using majority voting (50+1 %)
y_pred = (y_pred_df.sum(axis=1) >= (n_estimators / 2)).astype(np.int)

print('F1 score', metrics.f1_score(y_pred, y_test))
print('Accuracy score', metrics.accuracy_score(y_pred, y_test))

F1 score 0.6238660153524076
Accuracy score 0.6406666666666667


# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [16]:
probability = []

for c in range(2,300,1):
    y_pred = (y_pred_df.sum(axis=1) >= (n_estimators / c)).astype(np.int)
    f1_sc_result = metrics.f1_score(y_pred, y_test)
    probability.append([c,1/c,f1_sc_result])

probability.sort(key=lambda x: x[2], reverse = True)
#print(probability)
print('With a c =',probability[0][0], 'the probability that maximize the f1 score (', probability[0][2],') is ', probability[0][1])

With a c = 5 the probability that maximize the f1 score ( 0.7073552425665103 ) is  0.2


# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [17]:
samples_oob = []
# show the "out-of-bag" observations for each sample
for sample in samples:
    samples_oob.append(sorted(set(range(n_samples)) - set(sample)))

In [18]:
errors = np.zeros(n_estimators)

for i in range(n_estimators):
    y_pred_ = trees[i].predict(X_train.iloc[samples_oob[i]])
    errors[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_)

In [19]:
alpha = (1 - errors) / (1 - errors).sum()

weighted_sum_1 = ((y_pred_df) * alpha).sum(axis=1)

weighted_sum_1.head(5)

1483    0.849671
2185    0.847961
2520    0.730340
3721    0.877484
3727    0.144787
dtype: float64

In [20]:
y_pred = (weighted_sum_1 >= 0.5).astype(np.int)
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.6243016759776537, 0.6413333333333333)

# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [21]:
probability2 = []

for d in range(2,300,1):
    y_pred = (weighted_sum_1 >= 1/d).astype(np.int)
    result = metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)
    probability2.append([d,1/d,result])

probability2.sort(key=lambda x: x[2], reverse = True)
#print(probability2)
print('With a d =',probability2[0][0], 'the probability that maximize the f1 score (', probability2[0][2],') is ', probability2[0][1])

With a d = 5 the probability that maximize the f1 score ( (0.7055084745762711, 0.6293333333333333) ) is  0.2


# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score

In [22]:
X_train_2 = pd.DataFrame(index=X_train.index, columns=list(range(n_estimators)))

for i in range(n_estimators):
    X_train_2[i] = trees[i].predict(X_train)
    
X_train_2.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
2420,1,1,1,1,0,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1225,1,1,0,1,1,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
324,0,0,0,0,0,0,0,1,0,0,...,1,1,1,1,1,1,1,1,1,1
5863,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1
86,0,1,0,1,1,0,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1


In [23]:
#Estimate a logistic regression using as input the estimated classifiers

from sklearn.linear_model import LogisticRegressionCV
lr = LogisticRegressionCV()
lr.fit(X_train_2, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [24]:
#lr.coef_

In [25]:
#Modify the probability threshold such that maximizes the F1-Score

y_pred = (lr.predict_proba(y_pred_df)[:,1] >= 0.3).astype(bool)
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.707509881422925, 0.6053333333333333)

In [26]:
probability3 = []

for f in np.linspace(0.1,1,11):
    y_pred = (lr.predict_proba(y_pred_df)[:,1] >= f).astype(bool)
    f1_sc_result = metrics.f1_score(y_pred, y_test)
    probability3.append([f,f1_sc_result])

probability3.sort(key=lambda x: x[1], reverse = True)
#print(probability)
print('The probability that maximize the f1 score (', probability3[0][1],') is ', probability3[0][0])

The probability that maximize the f1 score ( 0.7057546145494027 ) is  0.37


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
